<a href="https://colab.research.google.com/github/jermwatt/asclepius_dev/blob/main/text2speech2text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies for text2speech and speech2text
!pip install git+https://github.com/huggingface/transformers sentencepiece datasets soundfile torchaudio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ww0ocjvc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ww0ocjvc
  Resolved https://github.com/huggingface/transformers to commit f1a1eb4ae10eac56dccfcd55d0f3a65772dc2741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.2 MB/s

# 1. setup functions

In [2]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from transformers import pipeline


def text2speech_setup():
    # load text2speech model
    processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
    model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
    vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

    # load xvector containing speaker's voice characteristics from a dataset
    embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
    speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

    # load speaker embeddings
    # speaker_embeddings = speaker_embeddings.to("cuda")
    return processor, model, vocoder, speaker_embeddings


def speech2text_setup():
    # instantiate pipe
    speech2text_pipe = pipeline(model="openai/whisper-large", device=0)
    return speech2text_pipe


# setup text2speech pipeline
processor, model, vocoder, speaker_embeddings = text2speech_setup()

# setup speech2text pipeline
speech2text_pipe = speech2text_setup()

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset cmu-arctic-xvectors downloaded and prepared to /root/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f. Subsequent calls will reuse this data.


# 2.  inference functions

In [3]:
# text2speech inference
def text2speech_inference(text,audio_file_name='test.wav'):
    inputs = processor(text=text, return_tensors="pt")
    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
    sf.write(audio_file_name, speech.numpy(), samplerate=16000)

# speech2text inference
def speech2text_inference(audio_file_name='test.wav', text_file_name='text.txt'):
    output = speech2text_pipe(audio_file_name, chunk_length_s=30)
    print(output)
    with open(text_file_name, 'w') as f:
        f.write(output['text'])


# 3.  Test inference

In [4]:
# test text2speech processor
text='Am I getting out of breath doing things that I can normally do without a problem?'
text2speech_inference(text=text, audio_file_name='test_1.wav')

# test speech2text processor
speech2text_inference(audio_file_name='test_1.wav', text_file_name='test_1.txt')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': ' Am I getting out of breath doing things that I can normally do without a problem?'}


In [5]:
# test text2speech processor
text='Has my weight gone up or down compared to yesterday?  If so, by how many pounds?'
text2speech_inference(text=text, audio_file_name='test_2.wav')

# test speech2text processor
speech2text_inference(audio_file_name='test_2.wav', text_file_name='test_2.txt')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': ' Has my weight gone up or down compared to yesterday? If so, by how many pounds?'}


In [6]:
# test text2speech processor
text='Can I do all the things I can normally do, such as get dressed on my own, make meals, or go for walks?'
text2speech_inference(text=text, audio_file_name='test_3.wav')

# test speech2text processor
speech2text_inference(audio_file_name='test_3.wav', text_file_name='test_3.txt')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': ' Can I do all the things I can normally do, such as get dressed on my own, make meals, or go for walks?'}
